# Notebook 00: Configuration de l'Environnement

Bienvenue dans le projet "MAKERS: Multi Agent Knowledge Exploration & Retrieval System" !

Ce notebook vous guidera à travers les étapes initiales de configuration de votre environnement de développement.

## Étapes Préalables

1.  **Installer Conda/Miniconda/Anaconda** : Si vous ne l'avez pas déjà, installez une distribution Conda adaptée à votre système d'exploitation depuis [Anaconda.com](https://www.anaconda.com/products/distribution) ou [Miniconda](https://docs.conda.io/en/latest/miniconda.html).

2.  **Créer l'Environnement Conda** :
    *   Ouvrez un terminal ou une invite de commande Anaconda.
    *   Naviguez jusqu'à la racine de ce projet (`makers/`).
    *   Exécutez la commande suivante pour créer l'environnement nommé `makers` à partir du fichier `environment.yml` fourni :
        ```bash
        conda env create -f environment.yml
        ```
    *   Si l'environnement existe déjà et que vous souhaitez le mettre à jour :
        ```bash
        conda env update -f environment.yml --prune
        ```
    *   *(Note: L'environnement Conda est recommandé pour le développement local. Pour le déploiement via Docker, le fichier `requirements.txt` est utilisé, qui peut être généré à partir de l'environnement Conda.)*

3.  **Activer l'Environnement Conda** :
    ```bash
    conda activate makers
    ```
    Vous devriez voir `(makers)` au début de votre invite de commande.

4.  **Créer le Fichier `.env`** :
    *   À la racine du projet (`makers/`), localisez le fichier `.env.example`. S'il n'existe pas, créez-le en vous basant sur la documentation du projet (README) pour les variables nécessaires.
    *   Créez une copie de `.env.example` et nommez-la `.env`.
    *   Remplissez les valeurs des variables d'environnement requises dans votre fichier `.env`, notamment :
        *   `MONGODB_URI` (votre chaîne de connexion MongoDB)
        *   `OLLAMA_BASE_URL` (si vous utilisez Ollama, ex: `http://localhost:11434`)
        *   Optionnellement, les clés API si vous utilisez des services externes comme `OPENAI_API_KEY`, `WANDB_API_KEY`, etc.

## Vérification de l'Environnement

Les cellules de code ci-dessous vous aideront à vérifier que tout est correctement configuré.

In [1]:
# Importer les bibliothèques nécessaires pour ce notebook de configuration
import os
from dotenv import load_dotenv
from pathlib import Path
import sys # Pour la modification de sys.path

print("Bibliothèques principales pour la vérification de la configuration importées avec succès.")

Bibliothèques principales pour la vérification de la configuration importées avec succès.


### 1. Chargement des Variables d'Environnement

Cette cellule charge les variables à partir de votre fichier `.env` et affiche certaines d'entre elles pour vérification (les clés API ne seront pas affichées intégralement).

In [2]:
# Déterminer le chemin vers le fichier .env à la racine du projet
# Les notebooks sont dans 'notebooks/', donc .env est dans le répertoire parent.
project_root_path = Path.cwd().parent # Path.cwd() dans un notebook est généralement le dossier du notebook.
dotenv_path = project_root_path / ".env"

if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION : Fichier .env non trouvé à l'emplacement attendu : {dotenv_path}")
    print("Veuillez vous assurer d'avoir créé et configuré votre fichier .env à la racine du projet (makers/).")
    print("Certaines configurations ci-dessous pourraient ne pas être chargées correctement.")

# Ajouter la racine du projet au PYTHONPATH pour permettre l'import de config.settings
if str(project_root_path) not in sys.path:
    sys.path.append(str(project_root_path))
    print(f"Racine du projet ({project_root_path}) ajoutée au PYTHONPATH.")

# Importer les settings de notre projet
try:
    from config.settings import settings
    print("Configuration du projet (config.settings.settings) chargée avec succès.")
except ImportError as e:
    print(f"ERREUR lors de l'import de config.settings: {e}")
    print("Assurez-vous que la racine du projet est dans PYTHONPATH et que le fichier .env est correct.")
    settings = None 

# Afficher les configurations clés
if settings:
    print(f"\n--- Vérification des Paramètres Essentiels ---")
    print(f"Nom du Projet: {settings.PROJECT_NAME}")
    print(f"Environnement Python (PYTHON_ENV): {settings.PYTHON_ENV}")

    print(f"\n--- LLM & Embedding ---")
    print(f"Fournisseur LLM Génératif par Défaut: {settings.DEFAULT_LLM_MODEL_PROVIDER}")
    if settings.DEFAULT_LLM_MODEL_PROVIDER == 'ollama':
        print(f"  Modèle Ollama Génératif: {settings.OLLAMA_GENERATIVE_MODEL_NAME}")
        print(f"  URL Base Ollama: {settings.OLLAMA_BASE_URL}")
    
    print(f"Fournisseur d'Embedding par Défaut: {settings.DEFAULT_EMBEDDING_PROVIDER}")
    if settings.DEFAULT_EMBEDDING_PROVIDER == 'ollama':
        print(f"  Modèle Ollama Embedding: {settings.OLLAMA_EMBEDDING_MODEL_NAME}")

    print(f"\n--- Clés API (Présence) ---")
    api_keys_to_check = {
        "OPENAI_API_KEY": settings.OPENAI_API_KEY,
        "HUGGINGFACE_API_KEY": settings.HUGGINGFACE_API_KEY,
        "WANDB_API_KEY": settings.WANDB_API_KEY,
        "ANTHROPIC_API_KEY": settings.ANTHROPIC_API_KEY,
        "GROQ_API_KEY": settings.GROQ_API_KEY,
        "TAVILY_API_KEY": settings.TAVILY_API_KEY
    }
    for key_name, key_value in api_keys_to_check.items():
        loaded_status = "Chargée" if key_value else "Non chargée/Vide"
        display_value = f" (début : {key_value[:4]}...)" if key_value else ""
        print(f"  {key_name}: {loaded_status}{display_value}")

    print(f"\n--- Configuration MongoDB ---")
    print(f"  MONGODB_URI configuré: {bool(settings.MONGODB_URI)}")
    if settings.MONGODB_URI and "localhost" not in settings.MONGODB_URI and "127.0.0.1" not in settings.MONGODB_URI:
         print(f"    (Note: Utilisation d'un URI MongoDB distant/Atlas)")
    elif settings.MONGODB_URI:
         print(f"    (Note: Utilisation d'un URI MongoDB local)")
    else:
        print(f"    ATTENTION: MONGODB_URI n'est pas défini. Les opérations avec la base de données échoueront.")
    print(f"  Nom de la Base de Données: {settings.MONGO_DATABASE_NAME}")

    print(f"\n--- Chemins des Données ---")
    print(f"  Répertoire DATA_DIR: {settings.DATA_DIR}")
    if settings.DATA_DIR.exists():
        print(f"    Le répertoire DATA_DIR existe.")
    else:
        print(f"    ATTENTION : Le répertoire DATA_DIR ({settings.DATA_DIR}) n'existe pas. Il sera créé par les scripts d'ingestion si nécessaire.")
    
    if settings.EVALUATION_DATASET_PATH:
        eval_path = Path(settings.EVALUATION_DATASET_PATH)
        print(f"  Dataset d'Évaluation: {eval_path}")
        if eval_path.is_file():
            print(f"    Le fichier du dataset d'évaluation existe.")
        else:
            print(f"    ATTENTION : Le fichier du dataset d'évaluation ({eval_path}) n'existe PAS ou n'est pas un fichier.")
    else:
        print("  Aucun chemin de dataset d'évaluation RAG n'est configuré.")
else:
    print("\nLes paramètres du projet (settings) n'ont pas pu être chargés. Veuillez vérifier les étapes précédentes.")


Variables d'environnement chargées depuis : /home/facetoface/makers/.env
Configuration du projet (config.settings.settings) chargée avec succès.

--- Vérification des Paramètres Essentiels ---
Nom du Projet: MAKERS: Multi Agent Knowledge Exploration & Retrieval System
Environnement Python (PYTHON_ENV): development

--- LLM & Embedding ---
Fournisseur LLM Génératif par Défaut: ollama
  Modèle Ollama Génératif: mistral
  URL Base Ollama: http://localhost:11434
Fournisseur d'Embedding par Défaut: ollama
  Modèle Ollama Embedding: nomic-embed-text

--- Clés API (Présence) ---
  OPENAI_API_KEY: Non chargée/Vide
  HUGGINGFACE_API_KEY: Non chargée/Vide
  WANDB_API_KEY: Non chargée/Vide
  ANTHROPIC_API_KEY: Non chargée/Vide
  GROQ_API_KEY: Non chargée/Vide
  TAVILY_API_KEY: Non chargée/Vide

--- Configuration MongoDB ---
  MONGODB_URI configuré: True
    (Note: Utilisation d'un URI MongoDB distant/Atlas)
  Nom de la Base de Données: makers_db

--- Chemins des Données ---
  Répertoire DATA_DIR:

### 2. Test de Connexion à MongoDB

Cette cellule tente de se connecter à votre instance MongoDB en utilisant l'URI fournie dans `.env` et chargée via `settings.py`.
Assurez-vous que votre base de données MongoDB Atlas est accessible (IP autorisée, etc.) ou que votre instance locale est en cours d'exécution.

In [3]:
# Importer pymongo (devrait être dans environment.yml)
try:
    from pymongo import MongoClient
    from pymongo.errors import ConnectionFailure, ConfigurationError
    print("Pymongo importé avec succès.")

    if settings and settings.MONGODB_URI:
        try:
            print(f"Tentative de connexion à MongoDB : {settings.MONGODB_URI[:20]}...") # Affiche seulement le début de l'URI
            # Utiliser un timeout court pour le test de connexion
            client = MongoClient(settings.MONGODB_URI, serverSelectionTimeoutMS=settings.MONGO_TIMEOUT_MS)
            
            # La commande ismaster est un moyen léger de vérifier la connexion.
            client.admin.command('ismaster') 
            print(f"Connexion à MongoDB réussie ! Version du serveur : {client.server_info()['version']}")
            
            # Optionnel : Lister les bases de données (si l'utilisateur a les droits)
            # try:
            #     print("Bases de données disponibles :", client.list_database_names()[:5]) # Affiche les 5 premières
            # except Exception as e_db_list:
            #     print(f"Impossible de lister les bases de données (droits insuffisants ou autre erreur) : {e_db_list}")

            client.close()

        except ConnectionFailure:
            print("ERREUR : Échec de la connexion à MongoDB. Vérifiez votre MONGODB_URI et l'accessibilité du serveur.")
            print("Causes possibles :")
            print("  - URI incorrecte dans le fichier .env.")
            print("  - Serveur MongoDB non démarré (si local).")
            print("  - Problème de réseau ou pare-feu bloquant la connexion.")
            print("  - Pour MongoDB Atlas : Adresse IP non autorisée dans les paramètres d'accès réseau.")
        except ConfigurationError as ce:
            print(f"ERREUR de configuration MongoDB : {ce}")
            print("Cela peut être dû à un format d'URI incorrect.")
        except Exception as e:
            print(f"Une erreur inattendue est survenue lors du test de connexion MongoDB : {e}")
    else:
        print("MONGODB_URI non configuré dans les settings. Test de connexion impossible.")

except ImportError:
    print("ERREUR : Pymongo n'est pas installé. Veuillez vérifier votre environnement Conda ('makers').")

Pymongo importé avec succès.
Tentative de connexion à MongoDB : mongodb+srv://user01...
Connexion à MongoDB réussie ! Version du serveur : 8.0.9


## Conclusion de la Vérification

Si toutes les cellules ci-dessus ont été exécutées et que les vérifications critiques (chargement des settings, connexion MongoDB si applicable) sont passées sans ERREUR majeure, votre environnement de base devrait être correctement configuré.

Quelques points d'ATTENTION peuvent subsister (ex: fichier de dataset d'évaluation manquant) mais ne bloqueront pas forcément tous les notebooks.

Vous pouvez maintenant procéder aux notebooks suivants pour explorer les fonctionnalités du projet MAKERS.